# Testing tool calling

In [ ]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Raw OpenAI client
client = OpenAI(
    api_key="YOU_API_KEY",
    base_url="https://api.lapathoniia.top"
)

# LangChain tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

def test_raw_openai():
    print("=== Testing Raw OpenAI Client ===")
    
    tools = [{
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer"},
                    "b": {"type": "integer"}
                },
                "required": ["a", "b"]
            }
        }
    }]
    
    response = client.chat.completions.create(
        model="lapa-function-calling",
        messages=[{"role": "user", "content": "Add 15 and 27 using the function"}],
        tools=tools,
        tool_choice="required"  # Force function calling
    )
    
    print(f"Response: {response.choices[0].message}")
    if response.choices[0].message.tool_calls:
        print("✓ Function called successfully!")
        for tool_call in response.choices[0].message.tool_calls:
            print(f"Function: {tool_call.function.name}")
            print(f"Arguments: {tool_call.function.arguments}")
            
            # Execute the function
            import json
            args = json.loads(tool_call.function.arguments)
            result = args['a'] + args['b']
            print(f"Function result: {result}")
    else:
        print("✗ No function calls made")

def test_langchain():
    print("\n=== Testing LangChain ===")
    
    llm = ChatOpenAI(
        model="gpt-5.2",
        api_key="",     
        # base_url="https://api.lapathoniia.top"
    )
    
    llm_with_tools = llm.bind_tools([add_numbers])#, tool_choice="any")
    
    response = llm_with_tools.invoke([HumanMessage(content="Use the add_numbers function to add 15 and 27")])
    
    print(f"Tool calls: {response.tool_calls}")
    if response.tool_calls:
        print("✓ Function called successfully!")
        for tool_call in response.tool_calls:
            print(f"Function: {tool_call['name']}")
            print(f"Arguments: {tool_call['args']}")
            
            # Execute the function
            result = add_numbers.invoke(tool_call['args'])
            print(f"Function result: {result}")
    else:
        print("✗ No function calls made")
        print(f"Text response: {response.content}")

In [2]:
test_langchain()


=== Testing LangChain ===
Tool calls: [{'name': 'add_numbers', 'args': {'a': 15, 'b': 27}, 'id': 'call_ran8wdNieqntmC5oQ1uWlcil', 'type': 'tool_call'}]
✓ Function called successfully!
Function: add_numbers
Arguments: {'a': 15, 'b': 27}
Function result: 42


In [4]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-X_GR0JPbgYsIVRspba2ArA",
    base_url="https://api.lapathoniia.top"
)

    
response = client.chat.completions.create(
    model="lapa-function-calling",
    messages=[{"role": "user", "content": "Як тебе звати?"}],
)

print(f"Response: {response.choices[0].message}")

Response: ChatCompletionMessage(content='Мене звати Gemini. Я великий мовний ШІ-модель, створений Google. Я можу допомагати з широким колом питань, давати розлогі відповіді та підтримувати інформативні розмови. Завжди радий допомогти!', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
